In [1]:
import pandas as pd
import numpy as np
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import unicodedata
import html

In [2]:
df = pd.read_excel('info.xlsx')

In [3]:
return_dict = {}
clean_dict ={}

In [165]:
def pledgeinfo(i):
    try:
        path = df['FName'].loc[i]
        link = 'https://www.sec.gov/Archives/' + path
        htmlfile = urllib.request.urlopen(link)
        htmltext = htmlfile.read().decode('utf-8')
        htmltext_1 = html.unescape(htmltext)
        soup = BeautifulSoup(htmltext_1, 'lxml')
        x = soup.get_text()
        text = x.split('. ')
        allmatch = [text[s] + '. ' for s in range (0, len(text)) if 'pledg' in ''.join(text[s:s+1])]
        pledge = ''.join(allmatch)
        if pledge != None:
            index = df['index'].loc[i]
            clean0 = pledge.replace('\n', " ")
            clean1 = clean0.replace('\t', " ")
            clean2 = " ".join(clean1.split())
            unicode_str = str(clean2)
            cleaned = unicodedata.normalize("NFKC", unicode_str)
            
            
            clean_dict[i] = (cleaned, index, path)
            print (i)
        else:
            print('clean error', i)
    except:
        return_dict[i] = ''
        print ('error', i)
        return

def getpledge(i):
    try:
        path = df['FName'].loc[i]
        link = 'https://www.sec.gov/Archives/' + path
        htmlfile = urllib.request.urlopen(link)
        htmltext = htmlfile.read().decode('utf-8')
        htmltext_1 = html.unescape(htmltext)
        soup = BeautifulSoup(htmltext_1, 'lxml')
        x = soup.get_text()
        return_dict[i] = x
        print (i)
    except:
        return_dict[i] = ''
        print ('error', i)
        return

def cleanpledge(i):
    if return_dict[i] != None:
        try:
            path = df['FName'].loc[i]
            index = df['index'].loc[i]
            clean0 = return_dict[i].replace('\n', " ")
            clean1 = clean0.replace('\t', " ")
            clean2 = " ".join(clean1.split())
            unicode_str = str(clean2)
            cleaned = unicodedata.normalize("NFKC", unicode_str)
            text = cleaned.split('. ')
            allmatch = [text[s] + '. ' for s in range (0, len(text)) if 'pledg' in ''.join(text[s:s+1])]
            pledge = ''.join(allmatch)
            clean_dict[i] = (pledge, index, path)
        except:
            print('clean error', i)
        return
    else:
        print('empty', i)
        return

def savepledge(i):
    getpledge(i)
    cleanpledge(i)
    return

In [5]:
    from joblib import Parallel, delayed
    import multiprocessing


In [156]:
def parallel(function, loop):
    core = multiprocessing.cpu_count()
    Parallel(n_jobs= core, backend='threading')(delayed(function)(i) for i in loop)

In [162]:
if 'pledg' in x:
    print('yes')

yes


In [166]:
parallel(pledgeinfo, range(0,10))

2
3
1
0
4
5
9
6
7
8


In [168]:
clean_dict[9]

('All shares covered by awards granted under this paragraph 13 shall be subject to the following restrictions during the Restricted Period: (i) The shares may not be sold, assigned, transferred, pledged, hypothecated or otherwise disposed of. (ii) Any additional common shares of the Company or other securities or property issued with respect to shares covered by awards granted under this paragraph 13 as a result of any stock dividend, stock split or reorganization, shall be subject to the restrictions and other provisions of this paragraph 13. (iii) A director shall not be entitled to receive any shares prior to completion of all actions deemed appropriate by the Company to comply with federal or state securities laws and stock exchange requirements. (f) Except in the event of conflict, all provisions of the Program shall apply to this paragraph 13.',
 19,
 'edgar/data/1800/0000912057-00-011369.txt')

In [ ]:
pledgedf = pd.DataFrame.from_dict(clean_dict, orient = 'index' )

In [ ]:
pledgedf.columns = ['infoclean2', 'index', 'path']

In [ ]:
pledgedf.to_excel('pledgeinfo.xlsx')